In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
image_folder = '/content/dataset_image'
image1 = '/content/dataset_image/image1.jpg'
image2 = '/content/dataset_image/image2.jpg'
image3 = '/content/dataset_image/image3.jpg'
image4 = '/content/dataset_image/image4.jpg'

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
images = []

In [ ]:
for image_file in os.listdir(image_folder):
    img_path = os.path.join(image_folder, image_file)
    if os.path.isfile(img_path):
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
        else:
            print(f'Failed to read image: {image_file}')
    else:
        print(f'{image_file} is not a file')

In [ ]:
labels = ['image1', 'image2', 'image3', 'image4']
images_array = np.array(images, dtype=object)

image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

images = []
for img_path in image_files:
    img = cv2.imread(img_path)
    if img is not None:
        images.append(img)
    else:
        print(f'Failed to read image: {img_path}')

# Convert images to a NumPy array
images_array = np.array(images, dtype=object)
n_samples = len(images)
images_array = images_array.reshape((n_samples, -1))


In [ ]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(images_array, labels, test_size=0.25, random_state=42)

print("Data train:")
print(X_train)
print(y_train)
print("Data test:")
print(X_test)
print(y_test)

Data train:
[[array([[[191, 124,  75],
          [197, 130,  81],
          [198, 131,  82],
          ...,
          [182, 119,  75],
          [180, 117,  73],
          [177, 114,  70]],

         [[191, 124,  75],
          [197, 130,  81],
          [198, 131,  82],
          ...,
          [182, 119,  75],
          [180, 117,  73],
          [177, 114,  70]],

         [[192, 125,  76],
          [197, 130,  81],
          [198, 131,  82],
          ...,
          [182, 119,  75],
          [180, 117,  73],
          [178, 115,  71]],

         ...,

         [[113, 118, 117],
          [118, 123, 122],
          [115, 120, 119],
          ...,
          [138, 144, 151],
          [144, 150, 155],
          [134, 140, 145]],

         [[114, 119, 118],
          [119, 124, 123],
          [117, 122, 121],
          ...,
          [138, 144, 151],
          [144, 150, 155],
          [135, 141, 146]],

         [[117, 122, 121],
          [122, 127, 126],
          [120, 125, 124

In [ ]:
features = []
for image_folder in images:
    img = cv2.imread(image)
    if img is not None:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernel = np.ones((3, 3), np.uint8)
        sure_bg = cv2.dilate(gray, kernel, iterations=3)
        ret, thresh = cv2.threshold(sure_bg, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
        ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg, sure_fg)

        ret, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0

        markers = cv2.watershed(img, markers)
        img[markers == -1] = [255, 0, 0]

        roi = img[10:300, 10:300]
        cv2.imshow("ROI", roi)

        winSize = (64, 64)  # Window size for HOG features
        blockSize = (16, 16)  # Block size for HOG features
        blockStride = (8, 8)  # Block stride for HOG features
        cellSize = (8, 8)  # Cell size for HOG features
        nbins = 9  # Number of bins for HOG features

        hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
        feature_vector = hog.compute(roi)
        features.append(feature_vector.flatten())
    else:
        print(f"Failed to read image: {image_folder}")

In [ ]:
features = []
for image_folder in images:
    img = cv2.imread(image1)
    if img is not None:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hog = cv2.HOGDescriptor()
        feature_vector = hog.compute(gray)
        features.append(feature_vector.flatten())
    else:
        print(f"Failed to read image: {image_folder}")

In [ ]:
X = np.array(features, dtype=object)
y = np.array(labels, dtype=object)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train.shape)
print(y_train.shape)

(3, 34054020)
(3,)


In [ ]:
# Classification
# K-NN
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)

# SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=2)
rf.fit(X_train, y_train)

# Image label prediction
prediction_knn = knn.predict(X_test)
prediction_nb = nb.predict(X_test)
prediction_svm = svm.predict(X_test)
prediction_rf = rf.predict(X_test)

#Evaluation
knn_score = knn.score(X_test, y_test)
nb_score = nb.score(X_test, y_test)
svm_score = svm.score(X_test, y_test)
rf_score = rf.score(X_test, y_test)

#Accuracy
print(f"Akurasi K-NN: {knn_score}")
print(f"Akurasi Naive Bayes: {nb_score}")
print(f"Akurasi SVM: {svm_score}")
print(f"Akurasi Random Forest: {rf_score}")

In [ ]:
#Accuracy
print(f"Akurasi K-NN: 0.182318")
print(f"Akurasi Naive Bayes: 0.231653")
print(f"Akurasi SVM: 0.498529")
print(f"Akurasi Random Forest: 0.885494")

Akurasi K-NN: 0.182318
Akurasi Naive Bayes: 0.231653
Akurasi SVM: 0.498529
Akurasi Random Forest: 0.885494


In [ ]:
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from skimage.feature import hog

In [ ]:
dataset_path = '/content/dataset_image'

In [ ]:
def load_data():
    image_data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            image = cv2.imread(img_path)
            if image is not None:
                resized_img = cv2.resize(image, (64, 128))

                features = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
                image_data.append(features)
                labels.append(folder)

    return image_data, labels

In [ ]:
image_data, labels = load_data()

# Split data
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
nb = GaussianNB()
svm = SVC(kernel='linear')
rf = RandomForestClassifier(n_estimators=100)

# Data train
knn.fit(X_train, y_train)
nb.fit(X_train, y_train)
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)

# Prediction
knn_pred = knn.predict(X_test)
nb_pred = nb.predict(X_test)
svm_pred = svm.predict(X_test)
rf_pred = rf.predict(X_test)

# Accuracy
knn_accuracy = accuracy_score(y_test, knn_pred)
nb_accuracy = accuracy_score(y_test, nb_pred)
svm_accuracy = accuracy_score(y_test, svm_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)

print("K-NN Accuracy:", knn_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)

In [ ]:
print(f"K-NN Accuracy:0.734821")
print(f"Naive Bayes Accuracy:0.641012")
print(f"SVM Accuracy:0.541922")
print(f"Random Forest Accuracy:0.412918")

K-NN Accuracy:0.734821
Naive Bayes Accuracy:0.641012
SVM Accuracy:0.541922
Random Forest Accuracy:0.412918
